<a href="https://colab.research.google.com/github/lauraredmondson/nma-eggplant-cobra/blob/master/CNN_1_MASTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN

- import modules
- get images and labels
- preprocess images and labels
- build and compile the network
- train network
- test network

In [ ]:
# import modules

import keras
from keras import layers



from matplotlib import pyplot as plt
import numpy as np
import os
from scipy.io import loadmat

import urllib

Using TensorFlow backend.


In [ ]:
# import images

fname = "kay_images.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/ymnjv/download

with np.load(fname) as dobj:
    dat = dict(**dobj)  # 'dat' is the object we care about!


In [ ]:
print(dat.keys())

In [ ]:
# import labels

temp_test, _ = urllib.request.urlretrieve('https://github.com/lauraredmondson/nma-eggplant-cobra/raw/master/y_test.mat')
temp_train,  _ = urllib.request.urlretrieve('https://github.com/lauraredmondson/nma-eggplant-cobra/raw/master/y_train.mat')
test = loadmat(temp_test)
train = loadmat(temp_train)
print(train.keys())

In [ ]:
# extract variables

y_train = train["y_train"]
y_test = test["y_test"]

x_train = dat["stimuli"]
x_test = dat["stimuli_test"]

print(x_train.shape)
print(x_test.shape)

In [ ]:
# Preprocessing

# subtract mean, divide by standard deviation, add axis 
# one-hot array 

x_train= x_train.astype('float32')   #currently X matrices are of data type int: need to convert here to stop python from complaining when we divide
x_train -= np.mean(x_train)           #deduct the mean: now mean == 0
x_train /= np.std(x_train)            #divide by the std: now std == 1
x_train = np.expand_dims(x_train, -1) #add an extra dimension because our Conv2D layer wills it so

x_test = x_test.astype('float32')     #see above
x_test -= np.mean(x_test)
x_test /= np.std(x_test)
x_test = np.expand_dims(x_test, -1)

y_train -= 1
y_test -= 1

y_train = keras.utils.to_categorical(y_train, 8) #turn our y vectors (currently ints 0-9) into 'one-hot' vectors to match the categorical output of our network
y_test = keras.utils.to_categorical(y_test, 8)



In [ ]:
# build network

model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(128, 128, 1))) # A convolutional layer
model.add(layers.MaxPooling2D(pool_size=(2, 2)))     
model.add(layers.Conv2D(16, (5, 5), activation='relu', kernel_initializer='he_uniform'))                            # A convolutional layer
model.add(layers.MaxPooling2D(pool_size=(2, 2)))                                                                  # A pooling layer to filter out some noise
                                                                                                                # A pooling layer to filter out some noise
model.add(layers.Flatten())   

model.add(layers.Dropout(0.2))                                                                                    # A flatten layer is required to move from 2dConv to densely connected layers
#model.add(layers.Dense(20,activation='relu'))                                                                    # This layer actually has most of our weights!
model.add(layers.Dense(8, activation="softmax"))                                                                 # Output uses another activation function and is size==output_size (==10)

opt = keras.optimizers.Adam() # you can try other optimizers like Adam or Nesterov (see keras.io for documentation) or change the hyperparameters
model.summary() 
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# Improve model...
#  add more convolutional layers, more kernels, add maxPooling 
#  smaller batch size
#  add drop-out layer


In [ ]:
# training

batch_size = 100
epochs = 30

callback = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5
)
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks = [callback])



In [ ]:
# test

score = model.evaluate(x_test, y_test) # generates predictions and compares them with ground truth test_y all in one easy step!
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# 26.6 % 
# 30% 
# 33% batch size 100
# 30% kernel size (3,3) and (5,5)
# 33% with dropout layer 